# Flights Data Exploration
## by (Medhat Hamdy Zaki)

## Preliminary Wrangling

>This document explores a dataset containing flights statistics by tracking  flights operated by large air carriers

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df_2006=pd.read_csv('2006.csv')
df_2007=pd.read_csv('2007.csv')
df_2008=pd.read_csv('2008.csv')

In [3]:
df=df_2006.append([df_2007,df_2008])

### What is the structure of your dataset?

There are 16,984,354 record in the dataset with 29 features Most variables are numeric in nature

In [4]:
# high-level overview of data shape and composition
print(df.shape)
print(df.dtypes)
df.head(5)

(16984354, 29)
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance               int64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
CancellationCode      object
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtype: object


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,...,45.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,...,27.0,19.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,...,4.0,11.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,...,16.0,10.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,...,27.0,12.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0


In [5]:
# descriptive ststistics for numeric variables
df.describe()

MemoryError: Unable to allocate 1.27 GiB for an array with shape (10, 16984354) and data type int64


### What is/are the main feature(s) of interest in your dataset?

I'm interested in figuring out the cause of delay and cancellation 

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

We have 5 factories as per the data set affecting both cancellation and delay (Carrier, Weather, NAS,Secuirity,LateAircraft) 

## Univariate Exploration

### I'll start by looking at which factor cause more Cancellation?

I will start off by removing the not cancelled flight

In [ ]:
#checking the unique value for carrier
df.UniqueCarrier.unique()

In [ ]:
# by using the carrier data set I replaced the names I need 
df['UniqueCarrier'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Pinnacle Airlines',
    'B6':'JetBlue Airways',
    'EV':'Atlantic Southeast Airlines',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'American Eagle',
    'NK':'Spirit Airlines',
    'OH':'Comair Inc.',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Midwest Airline',
    'AA':'American Airlines',
    'DL':'Delta Airlines',
    'US':'US Airways',
    'NW':'Northwest Airlines',
    'FL':'AirTran Airways',
    'XE':'Expressjet Airlines',
    'CO':'Continental Air Lines',
    'AQ':'Aloha Airlines ',
    'TZ':'ATA Airlines'
},inplace=True)

In [ ]:
df['Dest'].replace({
    'MQT':'Marquette County Airport',
    'ACK':'Nantucket Memorial',
    'CMX':'Houghton County Memorial',
    'EWR':'Newark Intl',
    'HKY':'Hickory Municipal',
    'MCN':'Middle Georgia Regional',
    'ORD':"Chicago O'Hare International",
    'HHH':'Hilton Head',
    'SPI':'Capital',
    'ILG':'New Castle County '
    
},inplace=True)

In [ ]:
df_can = df[df['Cancelled'] == 1]

In [ ]:
#Selecting the color palette
default_color = sns.color_palette()[0]

#set the size of the plot 
plt.figure(figsize=[11.69, 8.27])

g=sns.countplot(data=df_can, x='CancellationCode', color =default_color,order=['A','B','C','D']);
plt.xlabel('Cancellation Cause', fontsize='large')
plt.ylabel('Number Of Cancellation', fontsize='large')
plt.title('Count Per Cancellation Cause',fontsize=14, fontweight='bold')
g.set_xticklabels(['Carrier','Weather','National Air System','Security']);
plt.show()

- We Can see here based on the above plot most of cancellation came from Carrier cause.

### I will investigate more to check which carrier cause more cancellation ?

In [ ]:
# I will filter the data set based on carrier cause 
carrier_df = df_can[df_can['CancellationCode']=='A']

#then I will count the number of cancellation per carrier
carr_counts = carrier_df['UniqueCarrier'].value_counts()

carr_counts
# get the unique values in carrier in decreasing order 
carr_order = carr_counts.index

In [ ]:
#let's plot the count bar 
base_color = sns.color_palette()[0]
plt.figure(figsize=(11.69, 8.27))

sns.countplot(data=carrier_df, y='UniqueCarrier', color=base_color, order=carr_order);

for i in range (carr_counts.shape[0]):
    count = carr_counts[i]
    pct_string = '{:0.0f}'.format(count)
    plt.text(count+1, i, pct_string, va='center')
    
plt.ylabel('Carrier Name', fontsize='large')
plt.xlabel('Number Of Cancellation', fontsize='large')
plt.title('Count for cancellation cause per Carrier', fontsize=16, fontweight='bold');

- We can see that the top carrier causing cancellation based on only carrier factor are (United Air Lines Inc. (UA) , American Eagle Airlines Inc.(MQ),American Airlines Inc.(AA))

### I will check the distribution of the delay ?

- Since we are checking the delay here so we should exclude any negative values which indicate that there is no delay 
- Delayed flight will not be a cancelled so we will exclude the cancelled

In [ ]:
# Filter out the null values for arrdelay and DepDelay and nigative values

df_dely = df[(df['ArrDelay'] > 0) & (df['DepDelay'] > 0)]
df_dely = df_dely[df_dely['Cancelled'] == 0]

df_dely.shape

In [ ]:
df_dely.ArrDelay.isnull().sum()

In [ ]:
np.log10(df_dely['ArrDelay'].describe())

In [ ]:
np.log10(df_dely['DepDelay'].describe())

In [ ]:
# start with a standard-scaled plot to check the Arrival Delay
binsize = 5
bins = np.arange(0, df_dely['ArrDelay'].max()+binsize, binsize)

plt.figure(figsize=[11.69, 8.27])

plt.hist(data = df_dely, x = 'ArrDelay', bins = bins)
plt.xlabel('Arrival Delay (Min)', fontsize='large')
plt.ylabel('Counts', fontsize='large')
plt.title('Arrival Delay distribution', fontsize=16, fontweight='bold');
plt.xlim(0,270)
plt.show()

In [ ]:
# there's a long tail in the distribution, so let's put it on a log scale instead
log_binsize = 0.10
bins = 10 ** np.arange(0, np.log10(df_dely['ArrDelay'].max())+log_binsize, log_binsize)

plt.figure(figsize=[11.69, 8.27])

# Generate the x-ticks 
ticks = [5,10,20, 30, 100, 300, 1000]

# Convert ticks into string values, to be displaye dlong the x-axis
labels = ['{}'.format(v) for v in ticks]

plt.hist(data = df_dely, x = 'ArrDelay', bins = bins)
plt.xscale('log')

# Apply x-ticks
plt.xticks(ticks, labels);
plt.ylabel('Count', fontsize='large')
plt.xlabel('Arrival Delay (Min)', fontsize='large')
plt.title('Arrival Delay distribution -Log', fontsize=16, fontweight='bold');

plt.show()

Arrival Delay  has a long-tailed distribution The graph is skewed to the right. When plotted on a log-scale, the Arrdelay distribution looks roughly bimodal, with one peak at 5 mins, and a second peak between 10 and 30 min.

In [ ]:
# Now with a standard-scaled plot to check the Departure Delay
binsize = 2.5
bins = np.arange(0, df_dely['DepDelay'].max()+binsize, binsize)

plt.figure(figsize=[11.69, 8.27])

plt.hist(data = df_dely, x = 'DepDelay', bins = bins)
plt.xlabel('Departure Delay (Min)', fontsize='large')
plt.ylabel('Count', fontsize='large')
plt.title('Departure Delay distribution', fontsize=16, fontweight='bold');
plt.xlim(0,200)
plt.show()

In [ ]:
# there's a long tail in the distribution, so let's put it on a log scale instead
log_binsize = 0.10
bins = 10 ** np.arange(0, np.log10(df_dely['DepDelay'].max())+log_binsize, log_binsize)

plt.figure(figsize=[11.69, 8.27])

# Generate the x-ticks 
ticks = [5,10,20, 30, 100, 300, 1000]

# Convert ticks into string values, to be displaye dlong the x-axis
labels = ['{}'.format(v) for v in ticks]

plt.hist(data = df_dely, x = 'DepDelay', bins = bins)
plt.xscale('log')

# Apply x-ticks
plt.xticks(ticks, labels);
plt.ylabel('Count', fontsize='large')
plt.xlabel('Departure Delay (Min)', fontsize='large')
plt.title('Departure Delay distribution -Log', fontsize=16, fontweight='bold');

plt.show()

Departure Delay  has a long-tailed distribution The graph is skewed to the right. When plotted on a log-scale, the Depdelay distribution looks roughly bimodal, with one peak at 5 mins, and a second peak between 10 and 30 min.

### Which destination has the highest Delay?

In [ ]:
#Group by destinations codes and get sum of Cancellation for each destination
del_dest=df_dely.groupby('Dest').ArrDelay.mean().sort_values(ascending=False)
del_dest.head(10)

In [ ]:
# I decided  to Plot a bar chart having the proportions, instead of the actual count.

n_del = del_dest.value_counts().sum()

# Return the highest frequency in the `dest` column
max_del = del_dest[0]

# Return the maximum proportion, or in other words, 
# compute the length of the longest bar in terms of the proportion
max_prop = max_del / n_del
print(max_prop)

In [ ]:
# Use numpy.arange() function to produce a set of evenly spaced proportioned values 
# between 0 and max_prop, with a step size 2\%
tick_props = np.arange(0, max_prop, 0.02)

# Use a list comprehension to create tick_names that we will apply to the tick labels.
tick_names = ['{:0.2f}'.format(v) for v in tick_props]

In [ ]:
#let's plot the bar plot for only the highest 10 values 
base_color = sns.color_palette()[0]

plt.figure(figsize=(11.69, 8.27))

sns.barplot(x= del_dest.head(10).values, y=del_dest.head(10).index,color=base_color);

for i in range (del_dest.head(10).shape[0]):
    count = del_dest.values[i]
    pct_string = '{:0.1f}%'.format(100*count/n_del)
    plt.text(count+1, i, pct_string, va='center')

plt.xticks(tick_props * n_del, tick_names)
plt.xlabel('Proportion', fontsize='large');
plt.ylabel('Destination', fontsize='large')
plt.title('Proportion Of Delay Per Destination', fontsize=16, fontweight='bold');

- The highest destination delay are Marquette , Nantucket and Houghton between 21% to 23%

### Summary

- Started by looking into the cause of cancellation we can see that most of the cancellation were because of Carrier then Weather
- By investigating more into carrier cause we can see that (United Air Lines Inc. (UA), American Eagle Airlines Inc.(MQ), American Airlines Inc.(AA)) have the most cancellation 
- Then by looking into the delay distribution of both arrival and departure after excluding the negative values (early arrival or departure) we can see that the distribution has a long tail and the graph is skewed to the right so we plot it on a log scale we have found that in both graphs we have a peek at 5 Min and another peak between 10 to 30
- Finally, we checked the destination with the highest delay we can see that Marquette, Nantucket, and Houghton between 21% to 23%

## Bivariate Exploration

### what is the relation between arrival and departure delay?

In [ ]:
df_dely.columns

In [ ]:
# correlation plot
plt.figure(figsize = [11.69, 8.27])

# Scatter plot
sns.regplot(data = df_dely, x = 'ArrDelay', y = 'DepDelay',ci = None ,);

plt.ylabel('Departure Delay (min)', fontsize='large');
plt.xlabel('Arrival Delay (min)',  fontsize='large');
plt.title('Relation Between Arrival and Departure Time', fontsize=16, fontweight='bold');

The regression line in the scatter plot showing a positive correlation between the arrival and departure delay 

### Let's check the cancellation per month for every factor 

In [ ]:
# Convert the "Month" column from a plain object type into an ordered categorical type
months_order =[1,2,3,4,5,6,7,8,9,10,11,12]

# plot 
g=sns.FacetGrid(data=df_can, col='CancellationCode')
g.map(sns.countplot,'Month',order=months_order)

axes = g.axes.flatten()
axes[0].set_title("Carrier")
axes[2].set_title("Weather")
axes[1].set_title("NOS")
axes[3].set_title("Security");


It appears that most of Weather case cancellation happen in Feb and Dec and the most of carrier cancellation happen in the begining of the year, Most of security delay happen in Feb

### what is the mean arrival delay and departure delay per month

In [ ]:
# Arrival and departure delays per month of the year
plt.figure(figsize=(16, 5)).subplots_adjust(hspace = 0.5)

plt.subplot(1, 2 ,1)
ax1 = df_dely.groupby('Month').ArrDelay.mean().plot.bar().set_title('Arrival delays by month')
plt.title('Arrival Delays Per Month', fontsize=16, fontweight='bold')
plt.ylabel('Delay (Min)', fontsize='large')
plt.xlabel('Month', fontsize='large')
plt.xticks(rotation=0);

plt.subplot(1, 2 ,2)
df.groupby('Month').DepDelay.mean().plot.bar()
plt.title('Departure Delays Per Month', fontsize=16, fontweight='bold')
plt.ylabel('Delay (Min)', fontsize='large')
plt.xlabel('Month', fontsize='large')
plt.xticks(rotation=0);

plt.show()

Most of the highest delay happen in Jun,July and Dec

In [ ]:
# checking any other interested correlation 

df1 = df[['DepTime','ArrTime','ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay','Distance', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]


In [ ]:
# correlation plot
plt.figure(figsize = [16, 8])
sns.heatmap(df1.corr(), annot = True, fmt = '.3f',
           cmap = 'vlag_r', center = 0)

plt.title('Correlations Between Intersted Variables ',fontsize=14, fontweight='bold')

plt.show()

We can see that from the hitmap the distance correlated with (Airtime, Actual Elapsed Time and Estimated Elapsed Time) which make sense also we can see how Arrival dealy and Depature delay correlated with the 5 factories of study which we will stude in multivariate later

## Multivariate Exploration

### Which factor cause more delay per month ?

In [ ]:
df_dely.columns

In [ ]:
# I will create data by useing only the mean values of delay causes
df2 = df_dely[['Month','CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay','LateAircraftDelay']].groupby(['Month']).mean().reset_index()
df2

In [ ]:
# create a type and value colulms 
df2 = df2.melt(id_vars=['Month'], 
        var_name="DelayType", 
        value_name="Value")

In [ ]:
#Plot the bar graph
plt.figure(figsize=(11.69, 8.27))
ax = sns.barplot(data = df2, x = 'Month', y = 'Value', hue= 'DelayType',palette = 'Blues');
ax.legend(loc ='upper right', ncol = 3, framealpha = 1, title = 'DelayType')
plt.title('Monthly Delay Per Cause', fontsize=16, fontweight='bold')
plt.ylabel('Delay (Min)', fontsize='large')
plt.xlabel('Month', fontsize='large')

Most of the delay every month because of Late Aircraft Delay , Also we can see that there are a rare delay because of security

### Which Carrier has more delay at any month?

In [ ]:
# build the data frame 
df3 = df_dely[['Month','UniqueCarrier', 'CarrierDelay']].groupby(['Month','UniqueCarrier']).mean('CarrierDelay')
df3 = df3.reset_index()
df3 = df3.pivot(index = 'UniqueCarrier', columns = 'Month',values = 'CarrierDelay')
# plot
plt.figure(figsize=(11.69, 8.27))
sns.heatmap(df3, annot = True, fmt = '.3f',cbar_kws = {'label' : 'mean(Carrier Delay) (min)'},cmap="Blues")

plt.title('Monthly Delay Per Carrier ',fontsize=14, fontweight='bold')

plt.ylabel('Carrier Name', fontsize='large')
plt.xlabel('Month', fontsize='large')


From the graph above, we can see the delay happen because of carrier per each carrier for every month , we can see that Hawaiian airlines get the highest delay at October ,also we can see that Atlantic Southeast airlines was getting alot of delay all over the year 